# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [136]:
!cat publications.tsv

58089.14s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


pub_date	title	venue	authors	excerpt	abstract	citation	url_slug	pdf_url	doi	paper_url
2021-01-19	Climate-Driven Variability in Runoff Erosion Encoded in Stream Network Geometry	Geophysical Research Letters	Alexander Getraer and Adam C. Maloof		Climate signatures recorded in the geometry of branching streams provide insight into climate and landscape histories on Earth and other planetary bodies. Recent findings establish that branching angles are narrower and stream profiles are straighter in more arid climates. However, these two observations have been attributed to different mechanisms. Here we demonstrate that for US watersheds the difference in slope between confluent streams increases with humidity, and streams with a greater difference in slope tend to branch at wider angles. Our observations suggest a branching angle endmember of 90\u00B0 when stream slopes are most different. Using a simple model of runoff erosion, we show how this variation in relative stream slopes can be exp

## Import pandas

We are using the very handy pandas library for dataframes.

In [137]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [138]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0,encoding='unicode_escape')
publications


,pub_date,title,venue,authors,excerpt,abstract,citation,url_slug,pdf_url,doi,paper_url
0,2021-01-19,Climate-Driven Variability in Runoff Erosion E...,Geophysical Research Letters,Alexander Getraer and Adam C. Maloof,NaN,Climate signatures recorded in the geometry of...,"Getraer, A., & Maloof, A. C. (2021). Climate-D...",Getraer-Maloof-2021,http://agetraer.github.io/files/Getraer-Maloof...,10.1029/2020GL091777,https://doi.org/10.1029/2020GL091777
1,2024-12-20,Limits to timescale dependence in erosion rate...,Science Advances,"Joel A. Wilner, Bailey J Nordin, Alexander Get...",NaN,Earth's topography and climate result from the...,"Wilner, J. A., Nordin, B. J., Getraer, A., Gre...",Wilner-etal-2024,http://agetraer.github.io/files/Wilner-etal-20...,10.1126/sciadv.adr2009,https://doi.org/10.1126/sciadv.adr2009


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [139]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    "Produce entities within text."
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [148]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + "\""
    
    md += "\nauthors: " + item.authors

    md += "\ncollection: publications"

    md += "\npdfurl: " + item.pdf_url 

    md += "\npermalink: /publication/" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---\n\n------\n"
    
    ## Markdown description for individual page
    
    if len(str(item.abstract)) > 5:
        md += "\n>Abstract: <br/>" + html_escape(item.abstract) + "\n"
        
    md += "\n Suggested citation: <br/>" + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [149]:
!ls ../_publications/

61933.82s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


2021-01-19-Getraer-Maloof-2021.md 2024-12-20-Wilner-etal-2024.md
